In [21]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func, inspect
from sqlalchemy import Table, Column, Integer, String, Float, DateTime, MetaData

In [23]:
# Create an engine for the  database

engine = create_engine("sqlite:///../data/data.sqlite", echo=False)
conn = engine.connect()

In [24]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
meta = MetaData()

In [25]:
tweet_df = pd.read_csv('../data/tweet_only.csv')

In [26]:
tweet_list = tweet_df.text.tolist()

In [27]:
#- optional manual list of stop words.
stop_words = ['00', 'zzl2sc29dn', 'zzkh5cmnxy', 'zz41akot6q', 'zyrctsfmsq', 'zyezhe', \
             'zxurhtdtqq', 'zxl6syiwdm', 'zx9sj0quzn', 'http', 'https', '000', '000 000', 'at']

In [28]:
vectorizer = CountVectorizer(ngram_range=(2,4), stop_words='english') #- use built in stop words
#vectorizer = CountVectorizer(ngram_range=(2,3), stop_words=stop_words) #- use manual stop words

In [29]:
tf = vectorizer.fit_transform(tweet_list)

In [30]:
#- get results
res = pd.Series(np.ravel((vectorizer.transform(tweet_list).sum(axis=0))), index=vectorizer.get_feature_names())

In [31]:
df = pd.DataFrame({'phrase':res.index, 'cnt_phrase':res.values})

In [32]:
df.to_csv('../data/phrase_cnt.csv', index=False)

In [33]:
word_counts = Table(
    'phrases', meta, 
    Column('phrase_pkey', Integer, primary_key=True),
    Column('phrase',String), 
    Column('cnt_phrase', Integer)
)

In [34]:
meta.bind = engine
meta.drop_all()

In [35]:
meta.create_all()

In [36]:
pk_nums = list(range(1,len(df) + 1))

In [37]:
df.insert(0, "phrase_pkey", pk_nums)

In [38]:
df.to_sql('phrases',conn, if_exists='append', index=False)

In [39]:
# Compact/compress db after working on it.
engine.execute("VACUUM")

In [40]:
conn.close

<bound method Connection.close of <sqlalchemy.engine.base.Connection object at 0x0000025ABBAC3DD8>>